<a href="https://colab.research.google.com/github/nandhiniswaminathan/face-recognition/blob/master/vggface_resnet_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install keras_vggface
!pip install pandas
!pip install scikit_image

In [ ]:
import numpy as np
import pandas as pd
import sys,os
from skimage.transform import resize
from keras import backend as K
from keras.utils import to_categorical
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Flatten, Dense 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback
from keras import optimizers
from google.colab import drive
from keras.layers import Input
from keras.layers import Input, Dense, Conv2D, BatchNormalization, RepeatVector, Multiply, Permute, Reshape,MaxPooling2D, Flatten, Dropout, InputLayer,Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras import optimizers
import keras
from keras.regularizers import l2

Using TensorFlow backend.


In [ ]:
folder =  "/content/drive/Colab Notebooks/"

img_height, img_width = 197, 197

# Declaration of Parameters
num_classes         = 7     
epochs_top_layers   = 5
epochs_all_layers   = 50
batch_size          = 128
input_shape=(img_height, img_width,3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
filepath = "/content/drive/My Drive/Colab Notebooks/"
sys.path.insert(0, filepath)
os.chdir(filepath)

train_dir='train.csv'
test_dir='test.csv'

In [ ]:
wdata = pd.read_csv(train_dir)
data=wdata.iloc[1:1000,]
ddt=wdata.iloc[1001:1300,]
gd=pd.read_csv(test_dir)

In [ ]:
print(gd.size)
test=gd.iloc[0:1794,]
#for better accuracy train on more data 

7178


In [ ]:
base_model = VGGFace(
    model       = 'resnet50',
    include_top = False,
    weights     = 'vggface',
    input_shape = (img_height, img_width, 3))
base_model.summary()

94699520/94694792 [==============================] - 1s 0us/step
Model: "vggface_resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 99, 99, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 99, 99, 64)   256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           conv1/7x7_s2/bn[0][0]            
__________________

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(num_classes, activation = 'softmax')(x)

# The model to be trained
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
i = Input(shape=(197,197,3))
o1 = Conv2D(32, kernel_size=(3, 3), activation=None)(i)
o2 = BatchNormalization()(o1)
o3 = Conv2D(64, kernel_size=(3,3),activation=None)(o2)
o4 = BatchNormalization()(o3)
o1 = Conv2D(128,kernel_size=(3,3),activation=None)(o1)
o4 = Activation('relu')(o4)
o4 = Dropout(0.25)(o4)
o5 = keras.layers.add([o1,o3,o4])
o5 = BatchNormalization()(o5)
o5 = Activation('relu')(o5)

attn = Dense(1, activation='tanh')(o5)
attn = Flatten()(attn)
attn = Activation('softmax')(attn)
attn = RepeatVector(64)(attn)
attn = Permute([2, 1])(attn)

o5 = Reshape((193,64))(o5)
o6 = Multiply()([o5, attn])
o6 = Flatten()(o6)
o6 = Dense(12, activation=None)(o6)
o6 = BatchNormalization()(o6)
o6 = Activation('relu')(o6)
o6 = Dropout(0.5)(o6)
o6 = Dense(7,activation='softmax')(o6)


model_finetunedd = Model(inputs=i,outputs=o6)
print("model")



In [ ]:
model_finetuned = Sequential()
model_finetuned.add(model)
model_finetuned.add(Dense(32, activation='relu', input_dim=input_shape))
model_finetuned.add(Dropout(0.25))
model_finetuned.add(Dense(64, activation='relu'))
model_finetuned.add(Dropout(0.5))
model_finetuned.add(Dense(7, activation='sigmoid'))

model_finetuned.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])
model_finetuned.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 7)                 25666503  
_________________________________________________________________
dense_11 (Dense)             (None, 32)                256       
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 455       
Total params: 25,669,326
Trainable params: 25,616,206
Non-trainable params: 53,120
_____________________________________

In [ ]:
def preprocess_input(x):
    x -= 128.8006 # np.mean(train_dataset)
    return x

# Function to read the data from the csv file, increase the size of the images and return the images and their labels
def get_data(data):
    pixels = data['pixels'].tolist()
    images = np.empty((len(data), 48,48, 3))
    i = 0

    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(' ')]  # Extraction of each single
        single_image = np.asarray(single_image).reshape(48, 48) # Dimension: 48x48
        single_image = resize(single_image, (48,48), order = 3, mode = 'constant') # Dimension: 139x139x3 (Bicubic)
        ret = np.empty((48,48, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = to_categorical(data['emotion'])

    return images, labels    

# Data preparation
train_data_x, train_data_y  = get_data(data)
val_data  = get_data(ddt)


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
train_generator = train_datagen.flow(
    train_data_x,
    train_data_y,
    batch_size  = batch_size)

In [ ]:
model_finetuned.compile(
    optimizer   = Adam(lr = 1e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0), 
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])

In [ ]:
model_finetuned.fit_generator(
    generator           = train_generator,
    steps_per_epoch     = len(train_data_x) // batch_size,  # samples_per_epoch / batch_size
    epochs              = 1,                            
    validation_data     = val_data) 

In [ ]:
#fine-tuning the model by unfreezing the convolution layers
for layer in model.layers:
    layer.trainable = True

In [ ]:
from keras import optimizers
model.compile(
    optimizer   = optimizers.SGD(lr = 1e-4, momentum = 0.9, decay = 0.0, nesterov = True),
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])

In [ ]:
tensorboard_all_layers = TensorBoard(
    log_dir         = folder + '/logs_all_layers',
    histogram_freq  = 0,
    write_graph     = True,
    write_grads     = False,
    write_images    = True)
#save progress in drive

In [ ]:
def scheduler(epoch):
    updated_lr = K.get_value(model.optimizer.lr) * 0.5
    if (epoch % 3 == 0) and (epoch != 0):
        K.set_value(model.optimizer.lr, updated_lr)
        print(K.get_value(model.optimizer.lr))
    return K.get_value(model.optimizer.lr)

In [ ]:
# Learning rate scheduler
    # schedule: a function that takes an epoch index as input (integer, indexed from 0) and current learning
    #ate and returns a new learning rate as output (float)
reduce_lr = LearningRateScheduler(scheduler)

In [ ]:
reduce_lr_plateau = ReduceLROnPlateau(
	monitor 	= 'val_loss',
	factor		= 0.5,
	patience	= 3,
	mode 		= 'auto',
	min_lr		= 1e-8)

# Stop training when a monitored quantity has stopped improving
	# monitor:		Quantity to be monitored
	# patience:		Number of epochs with no improvement after which training will be stopped
	# mode: 		One of {auto, min, max}
early_stop = EarlyStopping(
	monitor 	= 'val_loss',
	patience 	= 10,
	mode 		= 'auto')

In [ ]:
model.fit_generator(
    generator           = train_generator,
    steps_per_epoch     = len(train_data_x) // batch_size,  # samples_per_epoch / batch_size 
    epochs              = 1,                        
    validation_data     = val_data,
    callbacks           = [reduce_lr, reduce_lr_plateau, early_stop])

Epoch 1/1
7/7 [==============================] - 260s 37s/step - loss: 24.7682 - accuracy: 0.1240 - val_loss: 859830.4114 - val_accuracy: 0.1639


In [ ]:
scores = model.evaluate(np.array(val_data), np.array(m), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))